In [1]:
import numpy as np
import argparse
import sys
import scipy.stats as st
import matplotlib.pyplot as plt
from pathlib import Path
# setting path
sys.path.append('/home/rafal/ROZKLADY/TopoTests/topotests/')
sys.path.append('/home/rafal/ROZKLADY/TopoTests/multiKS/')
sys.path.append('/home/rafal/ROZKLADY/TopoTests/2DKS/')

from topotests import TopoTest
from distributions import MultivariateDistribution, GaussianMixture, AbsoluteDistribution
from multiKS import multiKS
from KS2D import ks2d2s

import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import datetime
import sys, getopt
import scipy

import pickle

from ecc import *
import scipy.interpolate as spi

RayContext(dashboard_url='', python_version='3.9.7', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '192.168.1.200', 'raylet_ip_address': '192.168.1.200', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-15_23-43-35_114971_29185/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-15_23-43-35_114971_29185/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-05-15_23-43-35_114971_29185', 'metrics_export_port': 63269, 'gcs_address': '192.168.1.200:56244', 'address': '192.168.1.200:56244', 'node_id': '78bd247afcabb8fdebacc4f99e38915f4b0cd13b69b46a038f53356b'})

In [2]:
def get_ecc(X, xgrid, standarize=True):
    n = len(X)
    xmax = np.max(xgrid)
    ecc = np.array(compute_ECC_contributions_alpha(X))
    ecc[:, 1] = np.cumsum(ecc[:, 1])
    ecc = np.vstack([ecc, [xmax, 1]])
    if standarize:
        ecc[:, 1] = ecc[:, 1]/n
        ecc[:, 0] = ecc[:, 0]*n
    interpolator = spi.interp1d(ecc[:, 0], ecc[:, 1], kind='previous')
    ecc = interpolator(xgrid)
    return ecc

def dist_ecc(ecc1, ecc2):
    return np.max(np.abs(ecc1-ecc2))

In [3]:
rvs = [MultivariateDistribution([st.norm(), st.norm()], label='N01xN01'),
       MultivariateDistribution([st.norm(scale=2), st.norm(scale=2)], label='N02xN02'),
       MultivariateDistribution([st.t(df=3), st.t(df=3)], label='T3xT3'),
       MultivariateDistribution([st.t(df=5), st.t(df=5)], label='T5xT5'),
       MultivariateDistribution([st.t(df=10), st.t(df=10)], label='T10xT10'),
       MultivariateDistribution([st.logistic(), st.logistic()], label='LogisticxLogistic'),
       MultivariateDistribution([st.laplace(), st.laplace()], label='LaplacexLaplace'),
       MultivariateDistribution([st.norm(), st.t(df=5)], label='N01xT5'),
       MultivariateDistribution([GaussianMixture([-1, 1], [1, 1], [0.5, 0.5]),
                                GaussianMixture([-1, 1], [1, 1], [0.5, 0.5])], label='GM_1xGM_1'),
       MultivariateDistribution([st.norm(),
                                GaussianMixture([-1, 1], [1, 1], [0.5, 0.5])], label='N01xGM_1')      
      ]

In [4]:
def topotest2s(X1, X2, xgrid, mcloops=500):
    n = X1.shape[0]
    ecc1 = get_ecc(X1, xgrid=xgrid, standarize=False)
    ecc2 = get_ecc(X2, xgrid=xgrid, standarize=False)
    D = dist_ecc(ecc1, ecc2)
    X12 = np.vstack([X1, X2])
    distances = []
    for _ in range(mcloops):
        inds = np.random.permutation(2*n)
        X = X12[inds[:n]]
        y0 = get_ecc(X, xgrid=xgrid, standarize=False)
        X = X12[inds[n:]]
        y1 = get_ecc(X, xgrid=xgrid, standarize=False)
        distances.append(dist_ecc(y0, y1))
    pval = np.mean(distances > D)
    return D, pval, distances

In [5]:
def run_mc(rv1, rv2, n, mcloops_emp=500):
    xgrid = np.linspace(0, 3, 2500)
    pvals_tt = []
    pvals_ks = []
    for emp_loop in range(mcloops_emp):
        X1 = rv1.rvs(n)
        X2 = rv2.rvs(n)
        # run TopoTest
        D, pval_tt, distances = topotest2s(X1, X2, xgrid=xgrid)
        pvals_tt.append(pval_tt)
        # run K-S
        pvals_ks.append(ks2d2s(X1, X2)[1])
    return pvals_tt, pvals_ks

In [6]:
def run_all(n):
    import sys
    results = {}
    for rv1 in rvs:
        results[rv1.label] = {}
        print(f'n={n} label={rv1.label}')
        for rv2 in rvs:
            pvals_tt, pvals_ks = run_mc(rv1, rv2, n=n, mcloops_emp=10)
            results[rv1.label][rv2.label] = [n, pvals_tt, pvals_ks]
    with open(f'results.2d_2s/n{n}.pickle', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--n", type=int, required=True, help="sample size")
    args = parser.parse_args()

    run_all(args.n)

if __name__ == "__main__":
    main()